Week 6-TASK

File Ingestion and Schema Validation:
Take any csv/text file of 2+ GB of your choice. --- (You can do this assignment on Google colab)

Read the file ( Present approach of reading the file )

Try different methods of file reading eg: Dask, Modin, Ray, pandas and present your findings in term of computational efficiency

Perform basic validation on data columns : eg: remove special character , white spaces from the col name

As you already know the schema hence create a YAML file and write the column name in YAML file. --define separator of read and write file, column name in YAML

Validate number of columns and column name of ingested file with YAML.

Write the file in pipe separated text file (|) in gz format.

Create a summary of the file:

Total number of rows,

total number of columns

file size

In [1]:
import warnings
warnings.filterwarnings('ignore') 
import os

In [2]:
# Determine the size of the raw file:
os.path.getsize(r"D:\DataGlacier\Week6\test.csv")

3031783420

In [3]:
import os
file_path1 = r"D:\DataGlacier\Week6\test.csv"

def get_size(file_path1, unit='bytes'):
    file_size = os.path.getsize(file_path1)
    exponents_map = {'bytes': 0, 'kb': 1, 'mb': 2, 'gb': 3}
    if unit not in exponents_map:
        raise ValueError("Must select from \
        ['bytes', 'kb', 'mb', 'gb']")
    else:
        size = file_size / 1024 ** exponents_map[unit]
        return round(size, 3)


print(get_size(file_path1,'gb'),'GB')


2.824 GB


Read the files with various approaches:

In [4]:
import time
import csv
with open(r"D:\DataGlacier\Week6\test.csv") as csvfile:  
    df = csv.DictReader(csvfile)
start_time = time.time()
end_time = time.time()

In [5]:
#Read with DictReader

star_time=time.time()
dfdict = csv.DictReader(open(r"D:\DataGlacier\Week6\test.csv"))
a=("DictReader takes %s seconds" % (time.time() - start_time))
print(a)

DictReader takes 2.8767387866973877 seconds


In [6]:
#Read with Pandas

import pandas as pd
start_time = time.time()
end_time = time.time()
dfpandas = pd.read_csv(open(r"D:\DataGlacier\Week6\test.csv"))
b=("Pandas takes %s seconds" % (time.time() - start_time))
print(b)

Pandas takes 162.5535728931427 seconds


In [7]:
#Read with Dask

import dask.dataframe
start_time=time.time()
dfdask = dask.dataframe.read_csv(r"D:\DataGlacier\Week6\test.csv")
c=("DaskDataframe takes %s seconds" % (time.time() - start_time))
print(c)

DaskDataframe takes 0.06248784065246582 seconds


In [8]:
import modin.pandas as pd
import ray
ray.shutdown()
ray.init(runtime_env={'env_vars': {'__MODIN_AUTOIMPORT_PANDAS__': '1'}})
start_time=time.time()
dfmod=pd.read_csv(r"D:\DataGlacier\Week6\test.csv",chunksize=2000000)
for chunksize in dfmod:
#print(chunksize)
 d=("Modin and Ray takes %s seconds" % (time.time() - start_time))
print(d)


2023-04-12 10:34:58,824	INFO worker.py:1553 -- Started a local Ray instance.


Modin and Ray takes 82.9180121421814 seconds


In [9]:
print(a)
print(b)
print(c)
print(d)

DictReader takes 2.8767387866973877 seconds
Pandas takes 162.5535728931427 seconds
DaskDataframe takes 0.06248784065246582 seconds
Modin and Ray takes 82.9180121421814 seconds


In [10]:
print("The least time taken to read a csv file is: ", min(a,b,c,d))

The least time taken to read a csv file is:  DaskDataframe takes 0.06248784065246582 seconds


In [11]:
import dask.dataframe
df= dask.dataframe.read_csv(r"D:\DataGlacier\Week6\test.csv",delimiter=',')


In [12]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 11 entries, Timestamp to Is Laundering
dtypes: object(6), float64(2), int64(3)

In [13]:
df.describe()

,From Bank,To Bank,Amount Received,Amount Paid,Is Laundering
npartitions=1,,,,,
,float64,float64,float64,float64,float64
,...,...,...,...,...


Basic validation:

In [14]:
# Remove special characters
df.columns=df.columns.str.replace('[#,@,&,_]','')


In [15]:
# Remove white space from columns
df.columns = df.columns.str.replace(' ', '')

In [16]:
newdfdask=df.columns
newdfdask

Index(['Timestamp', 'FromBank', 'Account', 'ToBank', 'Account.1',
       'AmountReceived', 'ReceivingCurrency', 'AmountPaid', 'PaymentCurrency',
       'PaymentFormat', 'IsLaundering'],
      dtype='object')

In [17]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0



Overwriting testutility.py


In [18]:
%%writefile file.yaml
file_type: csv
dataset_name: testfile
file_name: test
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns:
     -Timestamp
     -FromBank
     -Account
     -ToBank
     -Account.1
     -AmountReceived
     -ReceivingCurrency
     -AmountPaid
     -PaymentCurrency
     -PaymentFormat
     -IsLaundering

Overwriting file.yaml


In [19]:
# Reading config file

import testutility as util
config_data = util.read_config_file('file.yaml')

In [20]:
#data of config file
config_data['inbound_delimiter']

','

In [21]:
config_data

{'file_type': 'csv',
 'dataset_name': 'testfile',
 'file_name': 'test',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': '-Timestamp -FromBank -Account -ToBank -Account.1 -AmountReceived -ReceivingCurrency -AmountPaid -PaymentCurrency -PaymentFormat -IsLaundering'}

In [22]:
# Reading process of the file using Dask
from dask import dataframe as dd
df_sample = dd.read_csv(r"D:\DataGlacier\Week6\test.csv",delimiter=',')
df_sample.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:17,20,800104D70,20,800104D70,6794.63,US Dollar,6794.63,US Dollar,Reinvestment,0
1,2022/09/01 00:02,3196,800107150,3196,800107150,7739.29,US Dollar,7739.29,US Dollar,Reinvestment,0
2,2022/09/01 00:17,1208,80010E430,1208,80010E430,1880.23,US Dollar,1880.23,US Dollar,Reinvestment,0
3,2022/09/01 00:03,1208,80010E650,20,80010E6F0,73966883.00,US Dollar,73966883.00,US Dollar,Cheque,0
4,2022/09/01 00:02,1208,80010E650,20,80010EA30,45868454.00,US Dollar,45868454.00,US Dollar,Cheque,0


In [23]:
 config_data['file_name']

'test'

In [33]:
source_file

'D:/DataGlacier/Week6/test.csv'

In [32]:
#Reading the file using config file
file_type = config_data['file_type']
#source_file = "D:/DataGlacier/Week6/" + config_data['file_name'] + f'.{file_type}'
source_file= "D:/DataGlacier/Week6/"+ config_data['file_name'] + f'.{file_type}'

In [34]:
import pandas as pd
df1 = pd.read_csv(source_file,config_data['inbound_delimiter'])
df1.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:17,20,800104D70,20,800104D70,6794.63,US Dollar,6794.63,US Dollar,Reinvestment,0
1,2022/09/01 00:02,3196,800107150,3196,800107150,7739.29,US Dollar,7739.29,US Dollar,Reinvestment,0
2,2022/09/01 00:17,1208,80010E430,1208,80010E430,1880.23,US Dollar,1880.23,US Dollar,Reinvestment,0
3,2022/09/01 00:03,1208,80010E650,20,80010E6F0,73966883.00,US Dollar,73966883.00,US Dollar,Cheque,0
4,2022/09/01 00:02,1208,80010E650,20,80010EA30,45868454.00,US Dollar,45868454.00,US Dollar,Cheque,0


In [37]:
#validating the header of the file
util.col_header_val(df1,config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['amount_paid', 'receiving_currency', 'from_bank', 'is_laundering', 'payment_format', 'to_bank', 'account_1', 'timestamp', 'amount_received', 'payment_currency', 'account']
Following YAML columns are not in the file uploaded ['a', 'c', 'o', 'g', '-', 'b', 't', ' ', 'v', 'u', 'k', '.', 'y', 'm', '1', 'd', 'p', 'r', 's', 'n', 'e', 'l', 'i', 'f']


0

In [38]:
print("columns of files are:" ,df1.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['timestamp', 'from_bank', 'account', 'to_bank', 'account_1',
       'amount_received', 'receiving_currency', 'amount_paid',
       'payment_currency', 'payment_format', 'is_laundering'],
      dtype='object')
columns of YAML are: -Timestamp -FromBank -Account -ToBank -Account.1 -AmountReceived -ReceivingCurrency -AmountPaid -PaymentCurrency -PaymentFormat -IsLaundering


In [39]:

if util.col_header_val(df1,config_data)==0:
    print("validation failed")
else:
    print("col validation passed")

column name and column length validation failed
Following File columns are not in the YAML file ['amount_paid', 'receiving_currency', 'from_bank', 'is_laundering', 'payment_format', 'to_bank', 'account_1', 'timestamp', 'amount_received', 'payment_currency', 'account']
Following YAML columns are not in the file uploaded ['a', 'c', 'o', 'g', '-', 'b', 't', ' ', 'v', 'u', 'k', '.', 'y', 'm', '1', 'd', 'p', 'r', 's', 'n', 'e', 'l', 'i', 'f']
validation failed


In [40]:
import datetime
import csv
import gzip

from dask import dataframe as dd
df = dd.read_csv(r'D:\DataGlacier\Week6\test.csv',delimiter=',')

# Write csv in gz format in pipe separated text file (|)
df.to_csv('test.csv.gz',
          sep='|',
          header=True,
          index=False,
          quoting=csv.QUOTE_ALL,
          compression='gzip',
          quotechar='"',
          doublequote=True,
          line_terminator='\n')

['C:/Users/NammaPC/File ingestion and schema validation/test.csv.gz\\00.part',
 'C:/Users/NammaPC/File ingestion and schema validation/test.csv.gz\\01.part',
 'C:/Users/NammaPC/File ingestion and schema validation/test.csv.gz\\02.part',
 'C:/Users/NammaPC/File ingestion and schema validation/test.csv.gz\\03.part',
 'C:/Users/NammaPC/File ingestion and schema validation/test.csv.gz\\04.part',
 'C:/Users/NammaPC/File ingestion and schema validation/test.csv.gz\\05.part',
 'C:/Users/NammaPC/File ingestion and schema validation/test.csv.gz\\06.part',
 'C:/Users/NammaPC/File ingestion and schema validation/test.csv.gz\\07.part',
 'C:/Users/NammaPC/File ingestion and schema validation/test.csv.gz\\08.part',
 'C:/Users/NammaPC/File ingestion and schema validation/test.csv.gz\\09.part',
 'C:/Users/NammaPC/File ingestion and schema validation/test.csv.gz\\10.part',
 'C:/Users/NammaPC/File ingestion and schema validation/test.csv.gz\\11.part',
 'C:/Users/NammaPC/File ingestion and schema validat

In [41]:
#number of files in gz format folder
import os
entries = os.listdir(r'C:/Users/NammaPC/File ingestion and schema validation/test.csv.gz')
for entry in entries:
    print(entry)

0.part
00.part
01.part
02.part
03.part
04.part
05.part
06.part
07.part
08.part
09.part
10.part
11.part
12.part
13.part
14.part
15.part
16.part
17.part
18.part
19.part
20.part
21.part
22.part
23.part
24.part
25.part
26.part
27.part
28.part
29.part
30.part
31.part
32.part
33.part
34.part
35.part
36.part
37.part
38.part
39.part
40.part
41.part
42.part
43.part
44.part
45.part
46.part
47.part


In [49]:
#size of the gz format folder
a=os.path.getsize(r'C:\Users\NammaPC\File ingestion and schema validation/test.csv.gz')
a

16384

In [50]:
import os
file_path = r'C:\Users\NammaPC\File ingestion and schema validation/test.csv.gz'

def get_size(file_path, unit='bytes'):
    file_size = os.path.getsize(file_path)
    exponents_map = {'bytes': 0, 'kb': 1, 'mb': 2, 'gb': 3}
    if unit not in exponents_map:
        raise ValueError("Must select from \
        ['bytes', 'kb', 'mb', 'gb']")
    else:
        size = file_size / 1024 ** exponents_map[unit]
        return round(size, 3)


sizeoffile=(get_size(file_path,'mb'))


In [51]:
print("Summary:")

# Total number of observations:
print("Total number of rows are:",len(df))

# Total number of features:
print("Total number of columns are :",len(df.columns))
df.columns

print("Size of the file is",sizeoffile,"GB")


Summary:
Total number of rows are: 31898238
Total number of columns are : 11
Size of the file is 0.016 GB
